# Bayesian Inference in FSharp


## Introduction

For my F# Advent submission this year, I decided to implement an extremely simple Bayesian Statistical Framework to be used for inference. The purpose of statistical inference is to infer characteristics of a population via samples to be used for further tasks such as making predictions. The "Bayesian" prefix refers to making use of the Bayes Theorem to conduct inferential statistics.

![image.png](./Images/StatisticalInference.png)

I like to think of Statistical Inference via a pasta cooking analogy: hypothetically, assume you lose the cooking instructions of dried pasta you are trying to cook. You start boiling the water and don't know when the pasta is al-dente. One way to check is to careful remove a single piece and test if it is cooked to your liking. Through this process, you'll know right away if the pasta needs to cook more or is ready to be sauce'd up. Similarly, inference deals with trying to figure out characteristics (__is the pasta ready?__) of a population (__all the pasta__) via a sample (__a single piece of pasta__).

![image.png](./Images/pasta.jpg)

F#, as a language, didn't fail to deliver an awesome development experience! The particular aspects of the language that made it easy to develop a Bayesian Statistical Framework are [Pattern Matching](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/pattern-matching) and Immutable Functional Data Structures such as [Records](https://fsharpforfunandprofit.com/posts/records/) and [Discriminated Unions](https://fsharpforfunandprofit.com/posts/discriminated-unions/) that make expressing the domain succinctly and lucidly not only for the developer but also the reader.


## Motivation

I recently received a Masters in Data Science but discovered that none of my courses dove deep into Bayesian Statistics. Therefore, motivated by a penchant desire to fully understand the differences between frequentist approaches and Bayesian ones to eventually gain an understanding of Bayesian Neural Networks was the impetus behind trying to understand the statistical theory. 

While reading  ["A First Course in Bayesian Statistical Methods"](https://www.amazon.com/Bayesian-Statistical-Methods-Springer-Statistics/dp/1441928286/ref=sr_1_2?crid=3D7COZ06U9AU1&dchild=1&keywords=a+first+course+in+bayesian+statistical+methods&qid=1608705310&sprefix=A+first+course+in+baye%2Caps%2C239&sr=8-2) by Peter Hoff (highly recommend this book to those with a strong mathematical background), I discovered [pymc3](https://docs.pymc.io/), a Probabilistic Programming library in Python. Pymc3, in my opinion, has some great abstractions that I wanted to implement myself in a functional-first setting. 

The best way to learn a new statistical algorithm is my implementing it and the best way to learn something well is to teach it to others and hence, this submission is a result of that ideology. 

## Bayes Theorem

The crux of Bayesian Inference lies in making use of the Bayes Theorem whose formula is the following:

$p(\theta \mid y) = \frac{p(y \mid \theta) p(\theta)}{p(y)}$

| Term           | Formula | Definition  |
| :------------- | :----------: | -----------: |
|  Prior         |  $p(\theta)$  | Quantification of t   |  |
| Likelihood     | $p(y \mid \theta)$ | In light of observations,  | | |
| Evidence       | $p(y)$ | The normalizing constant so that the probability is between 0 and 1. |
| Posterior      | $p(\theta \mid y)$ |  | | |

The premise of the interpretation of Bayes Theorem that resonates most with the scientific method is that you first come up with a hypothesis (prior), you figure out the likelihood that the hypothesis is valid based on available evidence and then quantify the validity of the hypothesis by highlighting the level of associated uncertainty. 



## Bayesian Inference in Practice

In practice, the denominator or the evidence i.e. the normalization constant of the Bayes theorem is a pain to calculate. For example, for continuous variables, Bayes theorem has the following form:

$p(\theta \mid y) = \frac{p(y \mid \theta) p(\theta)}{\int_{\Theta}p(y \mid \tilde{\theta}) p(\tilde{\theta}) \; d\tilde{\theta}}$

That integral in the denominator is UGLY and sometimes computational infeasible and so, approximation techniques must be used to get to generate the posterior distribution : enter __Markov-Chain Monte Carlo (MCMC)__ methods taking advantage of the fact that the denominator is a __normalization constant__ or in other words:

$p(\theta \mid y) \propto p(y \mid \theta) p(\theta)$


### Markov-Chain Monte-Carlo Methods

Monte Carlo  = Random Number Generation; you 
Markov-Chain = The future only depends on the present, not the past. Linked list of approximations.

## Goals

The goals of this submission are to develop the following:

1. __A Bayesian Domain Specific Language (DSL) and Its Parser__ that a statistician with no programming experience can understand and use to specify a model and its parameters and code that parses the model.
2. __A Bayesian Network__ from the Parsed Bayesian DSL addressing the simplest case i.e. one random variable representing the prior and one for the likelihood.
3. __Symmeteric Metropolis-Hastings__, an MCMC algorithm that makes use of the Bayesian DSL representation to approximate the posterior distribution.

![image](./Images/Plan.png)

## Getting Setup

The inferential logic will require minimal dependencies. Here are the dependencies and their respective uses:

| Dependency        | Use          |
| :-----------------| :----------: | 
| __XPlot.Plotly__  | Used for charting the histogram of the distributions.
| __MathNet.Numerics__     | Used for the out-of-the-box statistical distributions. |
| __Newtonsoft.Json__      | Parsing the parameters of the Bayesian DSL. |


In [2]:
// Dependencies
#r "nuget: XPlot.Plotly"
#r "nuget: MathNet.Numerics"
#r "nuget: Newtonsoft.Json"

// Imports 
open XPlot.Plotly
open MathNet.Numerics
open Newtonsoft.Json

## A Bayesian Domain Specific Language (DSL)  and Its Parser

![plan1](./Images/Plan_Section1.png)

The goal of this section is to develop "A Bayesian Domain Specific Language (DSL) that a statistician with no programming experience can understand and use to specify a model and its parameters and code that parses the model.". 

Drawing inspiration from [Stan](https://mc-stan.org) and pymc3 tutorials such as this [one](https://docs.pymc.io/notebooks/stochastic_volatility.html#Stochastic-Volatility-model), I wanted this DSL to be extremely simple but complete and therefore, the components I found that describe each random variable of the Bayesian Model are:

1. The Name of the Random Variable 
2. Conditionals of the Random Variables i.e. what other variables are given to be true to completely specify the random variable
3. The distribution associated with the random variable.
4. The parameters as variables of the distribution e.g. for a Normal Distribution, the parameters will be $\mu$ and $\sigma$, the mean and the variance respectively. 
5. The observed data for the Random Variable - if the random variable has 
6. A map of the parameters to constants to decouple the model from the values associated with the model.

### Format

```
Random Variable Name [|Comma-separated Conditionals] ~ Distribution(Comma-separated Parameters without spaces) [: observed] 
```

The details enclosed in ``[]`` imply optionality.

### Example

```
θ ~ Gamma(a,b)
Y|θ ~ Poisson(θ) : observed1
Z|Y,θ ~ Beta(θ, Y) : observed2
```

### The Domain

The domain or the types associated with the representation of the parsed random variable representing a single line in the specified model. 

In [3]:
#!fsharp

type ParsedRandomVariable = 
    { Name             : string
      Conditionals     : string list
      Distribution     : string
      Parameters       : string list
      Observed         : string option }
type ParsedBayesianModel = ParsedRandomVariable list

### Parsing Logic

The parsing logic for each line in the user specified model is as follows:
1. Split the line by spaces.
2. Extract the Name and the Conditionals from the first part of the model split on the tilde (~). ``Random Variable Name | [Conditionals]``.
3. From the second part of the model split on the tilde, get the name of the distribution and its associated parameters along with the optionally available observed variable.

F#'s awesome pattern matching was a lifesaver here not only for it's ease of use but it's automatic ability to specify the failure cases. 

In [4]:
#!fsharp
open System

// Format: RVName [|Conditionals] ~ Distribution(Parameters) [: observed] 
// [] -> optional
// NOTE: There can't be any spaces in the distribution parameters.
let parseLineOfModel (lineInModel : string) : ParsedRandomVariable = 
 
    // Helper fn to split the string based on a variety of type of delimiters.
    // Resultant type is a list of strings to feed in for the pattern matching.
    let splitToList (toSplit : string) (delimiters : obj) : string list = 
        let split = 
            match delimiters with
            | :? string        as s   -> toSplit.Split(s, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<string> as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<char>   as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | _ -> failwithf "Splitting based on delimiters failed as it is neither a string nor an array of strings: Of Type: %A - %A" (delimiters.GetType()) toSplit
        
        Array.toList split

    match splitToList lineInModel " " with
    | nameAndConditionals :: "~" :: distributionParametersObserved ->
        // Get the name and conditionals.
        let splitNameAndConditionals = splitToList nameAndConditionals "|"
        let name = splitNameAndConditionals.[0]
        let conditionals = 
            match splitNameAndConditionals with 
            | name :: conditionals -> 
                if conditionals.Length > 0 then splitToList conditionals.[0] ","
                else []
            | _ -> failwithf "Pattern not found for RV Name and Conditionals - the format is: RVName|Condtionals: %A" splitNameAndConditionals

        let extractAndGetParameters (distributionNameAndParameters : string) : string * string list = 
            let splitDistributionAndParameters = splitToList distributionNameAndParameters [| "("; ")" |]
            (splitDistributionAndParameters.[0], splitToList splitDistributionAndParameters.[1] ",")
            
        match distributionParametersObserved with 

        // Case: Without Observations. Example: θ ~ Gamma(a,b)
        | distributionNameAndParameters when distributionNameAndParameters.Length = 1 ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters.[0]
            { Name             = name; 
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = None; 
              Parameters       = snd extractedDistributionAndParameters; }

        // Case: With Observations. Example: Y|θ ~ Poisson(θ) : observed
        | distributionNameAndParameters :: ":" :: observed ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters
            { Name             = name;
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = Some observed.Head; // Only 1 observed list permitted.
              Parameters       = snd extractedDistributionAndParameters; } 

        // Case: Error.
        | _ -> failwithf "Pattern not found for the model while parsing the distribution, parameters and optionally, the observed variables: %A" distributionParametersObserved 

    | _ -> failwithf "Pattern not found for the following line in the model - please check the syntax: %A" lineInModel

let parseModel (model : string) : ParsedBayesianModel = 
    model.Split('\n') 
    |> Array.map(parseLineOfModel)
    |> Array.toList

let printParsedModel (model : string) : unit = 
    let parsedModel = parseModel model
    printfn "Model: %A is represented as %A" model parsedModel

#### Examples of Parsing a User-Specified DSL

In [7]:
// Print out our simple 1-Parameter Model.
let model1 = @"θ ~ Gamma(a,b)
              Y|θ ~ Poisson(θ) : observed"
printParsedModel(model1)

// This model doesn't make sense but adding to test multiple conditionals.
let model2  = @"θ ~ Beta(unit,unit)
               γ ~ Gamma(a,b)
               Y|θ,γ ~ Binomial(n,θ) : observed"
printParsedModel(model2)

Model: "θ ~ Gamma(a,b)
              Y|θ ~ Poisson(θ) : observed" is represented as [{ Name = "θ"
   Conditionals = []
   Distribution = "gamma"
   Parameters = ["a"; "b"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["θ"]
                        Distribution = "poisson"
                        Parameters = ["θ"]
                        Observed = Some "observed" }]
Model: "θ ~ Beta(unit,unit)
               γ ~ Gamma(a,b)
               Y|θ,γ ~ Binomial(n,θ) : observed" is represented as [{ Name = "θ"
   Conditionals = []
   Distribution = "beta"
   Parameters = ["unit"; "unit"]
   Observed = None }; { Name = "γ"
                        Conditionals = []
                        Distribution = "gamma"
                        Parameters = ["a"; "b"]
                        Observed = None }; { Name = "Y"
                                             Conditionals = ["θ"; "γ"]
                                             Distribution = "binomial"
              

### Specifying the Parameters

The idea is to decouple the parameters separate from the model and this is done by saving the details as a JSON string. This task is made simple using the ``Newtonsoft.Json`` library.

In [6]:
open System
open System.Collections.Generic

open MathNet.Numerics.Distributions

open Newtonsoft.Json

type Observed = float list

type ParameterList = 
    { Observed : float list; Parameters : Dictionary<string, float> } 

let deserializeParameters (paramsAsString : string) : ParameterList = 
    JsonConvert.DeserializeObject<ParameterList>(paramsAsString)

#### Examples of the Deserialization of the Parameters

In [ ]:
// Parameter List 1
let parameters1 = "{Parameters : {μ0 : 0, σ0 : 1, μ : 5, σ : 2, λ : 4}, observed : [4.2,0.235,2.11]}"
let deserializedParameters1 = deserializeParameters parameters1
printfn "Deserialized Parameters 1: %A" (deserializedParameters1)

// Parameter List 2
let parameters2 = "{Parameters: {λ : 2}}"
let deserializedParameters2 = deserializeParameters parameters2
// Applying the Deserialized Parameters to Sample from a Distribution
let exp = Exponential deserializedParameters2.Parameters.["λ"] 
printfn "Sampling from the Exponential Distribution with the λ = %A parameter: %A" exp (exp.Sample())

# Converting the Parsed Model into a Bayesian Network

![plan_section2](./Images/Plan_Section2.png)

The goal is to take the parsed representations of the user-inputted model to convert to a directed acylic graph that represents a Bayesian Network so that the posterior distribution can be computed easily. 

The first step here is to map the string version of a distribution to be used 

### Distribution Mapping

In [ ]:
#!fsharp
open MathNet.Numerics.Distributions

type DistributionType = 
    | Continuous
    | Discrete

type Parameter     = float
type DiscreteInput = int
type Input         = float

type DensityType = 
    | OneParameter         of (Parameter * Input -> float) 
    | OneParameterDiscrete of (Parameter * DiscreteInput -> float)
    | TwoParameter         of (Parameter * Parameter * Input -> float)

type DistributionInfo = { RVName             : string
                          DistributionType   : DistributionType 
                          DistributionName   : string 
                          Parameters         : float list
                          Density            : DensityType } with
                          
    static member Create (item : ParsedRandomVariable) 
                         (parameterList : ParameterList) : DistributionInfo = 
    // I know this is ugly but this functionality assumes the user enters the 
    // parameters in the order that's expected by the MathNet Numerics Library. 
    // Grab the parameters associated with this Random Variable.
    let rvParameters = 
        item.Parameters
        |> List.filter(parameterList.Parameters.ContainsKey) 
        |> List.map(fun item -> parameterList.Parameters.[item])

    // Extract Distribution Associated with the Parsed Random Variable.
    match item.Distribution with

    // 1 Parameter Distributions 
    | "exponential" -> 
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = OneParameter Exponential.PDF }
    | "poisson" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Discrete
          Density          = OneParameterDiscrete Poisson.PMF } 
    // 2 Parameter Distributions 
    | "normal" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = TwoParameter Normal.PDF }
    | "gamma" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = TwoParameter Gamma.PDF }
    | "beta" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = TwoParameter Beta.PDF }
    | "continuousuniform" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = TwoParameter ContinuousUniform.PDF }
    // Failure Case
    | _ -> failwithf "Distribution not registered: %A" item.Distribution                      
                          
    member this.ComputeOneParameterPDF (parameter : float) (input : float) : float =
        match this.Density with
        | OneParameter pdf -> pdf(parameter,input)
        | _ -> failwithf "Incorrect usage of function with a non One Parameter Density Type. Density given: %A" this.Density 
    member this.ComputeOneParameterDiscretePMF (parameter : float) (input : int) : float =
        match this.Density with
        | OneParameterDiscrete pmf -> pmf(parameter,input)
        | _ -> failwithf "Incorrect usage of function with a non One Parameter Discrete Density Type. Density given: %A" this.Density 
    member this.ComputeTwoParameterPDF (parameter1 : float) (parameter2 : float) (input : float) : float =
        match this.Density with
        | TwoParameter pdf -> pdf(parameter1, parameter2, input)
        | _ -> failwithf "Incorrect usage of function with a non Two Parameter Density Type. Density given: %A" this.Density

### Helper Methods to get the Distributions from the User Specified DSL

In [ ]:
let getDistributionInfoForModel(model : string) (parameterList : string) : DistributionInfo list = 
    let parsedModel   = parseModel model
    let parameterList = deserializeParameters parameterList 
    parsedModel
    |> List.map(fun x -> DistributionInfo.Create x parameterList)

let getDensityOrProbabilityForModel (model : string) 
                                    (parameterList : string) 
                                    (data : float seq) : IDictionary<string, float seq> = 
    getDistributionInfoForModel model parameterList
    |> List.map(fun (e : DistributionInfo) -> 
        match e.Density with
        | OneParameter p ->
            let param   = List.exactlyOne e.Parameters 
            let results = data |> Seq.map(fun d -> e.ComputeOneParameterPDF param d)
            e.RVName, results
        | OneParameterDiscrete p ->
            let param   = List.exactlyOne e.Parameters 
            let results = data |> Seq.map(fun d -> e.ComputeOneParameterDiscretePMF param (int d))
            e.RVName, results
        | TwoParameter p ->
            let p2 : float list = e.Parameters |> List.take 2
            let results = data |> Seq.map(fun d -> e.ComputeTwoParameterPDF p2.[0] p2.[1] d)
            e.RVName, results)
    |> dict

#### Testing the Distribution Mapping Logic

In [ ]:
#!fsharp

// Exponential. 
let exponentialModel     = "x ~ Exponential(lambda)"
let exponentialParamList = "{Parameters: {lambda : 2, a : 2., b : 2.3}, Observed : []}"
let exponentialDummyData = ContinuousUniform.Samples(0., 200.) |> Seq.take 2000
let exponentialPdfs      = getDensityOrProbabilityForModel exponentialModel exponentialParamList exponentialDummyData
printfn "Exponential: %A" (exponentialPdfs.Values)

// Normal.
let normalModel     = "x ~ Normal(mu,sigma)"
let normalParamList = "{Parameters: {mu: 0., sigma : 1.}, Observed : []}"
let normalDummyData = Normal.Samples(0.0, 1.0) |> Seq.take 2000
let normalPdfs      = getDensityOrProbabilityForModel normalModel normalParamList normalDummyData 
printfn "Normal: %A" (normalPdfs.Values)

// Poisson.
let poissonModel     = "x ~ Poisson(theta)"
let poissonParamList = "{Parameters: {theta: 44}, Observed : []}"
let poissonDummyData = ContinuousUniform.Samples(0., 5.) |> Seq.take 2000 
let poissonPdfs      = getDensityOrProbabilityForModel poissonModel poissonParamList poissonDummyData 
printfn "Poisson: %A" (poissonPdfs.Values)

### Bayesian Node

In [ ]:
#!fsharp
type BayesianNodeTypeInfo =
| Observed of float list 
| NonObserved

type BayesianNode = 
    { Name                 : string
      NodeType             : BayesianNodeTypeInfo 
      DistributionInfo     : DistributionInfo
      ParsedRandomVariable : ParsedRandomVariable } with

    static member ConstructNode(parsedRandomVariable : ParsedRandomVariable)
                               (parameterList : ParameterList) =

        let nodeType : BayesianNodeTypeInfo =
            match parsedRandomVariable.Observed with
            | Some _ -> BayesianNodeTypeInfo.Observed parameterList.Observed
            | None   -> BayesianNodeTypeInfo.NonObserved
        { Name                 = parsedRandomVariable.Name;
          NodeType             = nodeType;
          DistributionInfo     = DistributionInfo.Create parsedRandomVariable parameterList;
          ParsedRandomVariable = parsedRandomVariable; }

    member this.GetDependents (parsedBayesianModel : ParsedBayesianModel) : ParsedRandomVariable list =
        parsedBayesianModel 
        |> List.filter(fun x -> x.Conditionals |> List.contains(this.Name))

#### Construct a Node

In [ ]:
#!fsharp
let lineOfModel        = @"x ~ Exponential(lambda) : observed"
let paramList          = "{Parameters: {lambda : 2, a : 2., b : 2.3 }, observed : [1,2,3,55]}"
BayesianNode.ConstructNode (parseLineOfModel lineOfModel) (deserializeParameters paramList)

### The Simple Bayesian Network

In [ ]:
#!fsharp
// Only to be used for a model with 2 nodes 
// i.e. one for the prior and one for the likelihood.
type SimpleBayesianNetworkModel = 
  { Name       : string
    Nodes      : IDictionary<string, BayesianNode>
    Prior      : BayesianNode
    Likelihood : BayesianNode } with

    member this.GetPriorProbability (input : float) : float = 
      let distributionInfo = this.Prior.DistributionInfo 
      match distributionInfo.Density with
      | OneParameter p ->
          let param = List.exactlyOne distributionInfo.Parameters 
          distributionInfo.ComputeOneParameterPDF param input 
      | OneParameterDiscrete p ->
          let param   = List.exactlyOne distributionInfo.Parameters 
          distributionInfo.ComputeOneParameterDiscretePMF param (int input)
      | TwoParameter p ->
          let p2 : float list = distributionInfo.Parameters |> List.take 2
          distributionInfo.ComputeTwoParameterPDF p2.[0] p2.[1] input

    member this.GetLikelihoodProbability (prior : float) : float = 
        let distributionInfo = this.Likelihood.DistributionInfo
        let observed         = match this.Likelihood.NodeType with
                               | Observed l -> l
                               | _ -> failwithf "Incorrectly constructed Simple Network Model. %A" this 
        let density = 
          match distributionInfo.Density with
          | OneParameter p -> 
              observed |> List.map(fun d -> distributionInfo.ComputeOneParameterPDF prior d)
          | OneParameterDiscrete p ->
              observed |> List.map(fun d -> distributionInfo.ComputeOneParameterDiscretePMF prior (int (Math.Ceiling d)))
          | TwoParameter p ->
              let p : float = distributionInfo.Parameters |> List.exactlyOne 
              observed |> List.map(fun d -> distributionInfo.ComputeTwoParameterPDF prior p d)
        density
        |> List.fold (*) 1.0

    member this.GetPosteriorWithoutScalingFactor (input: float) (printDebug: bool) : float = 
      let priorPdf      = this.GetPriorProbability input 
      let likelihoodPdf = this.GetLikelihoodProbability priorPdf
      if printDebug then
          printfn "Prior: %A | Likelihood: %A" priorPdf likelihoodPdf
      priorPdf * likelihoodPdf

    static member ConstructModel (name : string)
                                 (model : ParsedBayesianModel)
                                 (parameterList : ParameterList) : SimpleBayesianNetworkModel = 

        // Construct all the modes of the model.
        let allNodes : (string * BayesianNode) list =
            model
            |> List.map(fun m -> m.Name, BayesianNode.ConstructNode m parameterList)

        let prior : BayesianNode =  
            allNodes
            |> List.filter(fun (_,m) -> m.NodeType = BayesianNodeTypeInfo.NonObserved) 
            |> List.map(fun (_,m) -> m)
            |> List.exactlyOne

        let likelihood : BayesianNode =  
            allNodes
            |> List.filter(fun (_,m) -> m.NodeType <> BayesianNodeTypeInfo.NonObserved)
            |> List.map(fun (_,m) -> m)
            |> List.exactlyOne

        { Name       = name; 
          Nodes      = dict allNodes; 
          Prior      = prior;
          Likelihood = likelihood; } 

#### Example of Creating a Simple Bayesian Network

In [ ]:
#!fsharp
let model = @"x ~ Normal(μ,τ) 
              y|x ~ Normal(x,σ) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {μ : 5, τ : 3.1622, σ : 1}, observed : [9.37,10.18,9.16,11.60,10.33]}"
let normalModel = 
    SimpleBayesianNetworkModel.ConstructModel "Normal-Normal" parsedModel (deserializeParameters paramList)
normalModel

#### Debugging the Prior and Likelihood

In [ ]:
#!fsharp
let model = @"x ~ Normal(μ,τ) 
              y|x ~ Normal(x,σ) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {μ : 5, τ : 3.1622, σ : 1}, observed : [9.37,10.18,9.16,11.60,10.33]}"
let normalModel = 
    SimpleBayesianNetworkModel.ConstructModel "Normal-Normal" parsedModel (deserializeParameters paramList)
    
// Prior Check
let priorFromModel = normalModel.GetPriorProbability 0.0
let computedPrior  = Normal.PDF(5., 3.1622, 0.0)
printfn "Normal Model Prior: %A | Prior from formula: %A" priorFromModel computedPrior

// Likelihood Check
let likelihoodFromModel = normalModel.GetLikelihoodProbability 0.0
let observed : float list = 
    match normalModel.Likelihood.NodeType with
    | Observed (o: float list) -> o
    | _ -> failwithf "Incorrectly constructed Likelihood"
    
let likelihoodFromFormula : float list = observed |> List.map(fun (o:float) -> Normal.PDF(computedPrior, 1., o))
printfn "%A" likelihoodFromFormula

printfn "Normal Model Likelihood: %A | Likelihood from formula: %A" (normalModel.GetLikelihoodProbability 0.03614314702) 

# Applying the Symmetric Metropolis Hastings Algorithm to Approximate the Posterior Distribution

![plan_section3](./Images/Plan_Section3.png)

### MCMC Domain

In [ ]:
#!fsharp
type ConvergenceCriteria = 
    | IterativeConvergence of int

type ProposalDistribution =
    | Normal of float  // Normal( current, delta )
    | ContinuousUniform of float // ContinuousUniform( current - delta, current + delta )
    | PositiveContinuousUniform of float // ( x ~ Uniform( current - delta, current + delta ) if x <= 0 then 0.1 else x )

type MCMCInferenceStep = 
    | SymmetricMetropolisHastings of ProposalDistribution * float

type MCMCChain =
    { Id             : int
      AcceptanceRate : float
      StepValues     : float seq }

type MCMCRequest = 
    { StepFunction         : MCMCInferenceStep
      ConvergenceCriteria  : ConvergenceCriteria
      BurnInIterationsPct  : float 
      Chains               : int 
      PrintDebug           : bool }

type MCMCResult =
    { Chains         : MCMCChain seq 
      MCMCRequest    : MCMCRequest }

### The Symmetric Metropolis Hastings Algorithm

In [ ]:
#!fsharp
open System
open MathNet.Numerics.Distributions
open XPlot.Plotly

type AcceptanceRejection =
    | Acceptance of float
    | Rejection  of float 

let doSymmetricMetropolisHastings (request : MCMCRequest) 
                                  (iterations : int)
                                  (proposalDistribution : ProposalDistribution)
                                  (initialValue : float)
                                  (simpleBayesianModel : SimpleBayesianNetworkModel) : MCMCResult =

    let getChain (id : int) (request : MCMCRequest) 
                 (iterations : int) (simpleBayesianModel : SimpleBayesianNetworkModel) : MCMCChain =

        let burnin         = int (Math.Ceiling(request.BurnInIterationsPct / 100. * float iterations))
        let zeroOneUniform = ContinuousUniform()

        let mutable current = simpleBayesianModel.GetPosteriorWithoutScalingFactor initialValue request.PrintDebug

        let matchedProposalDistribution (input : float) :  float =
            match proposalDistribution with 
            | Normal delta            -> Normal(input, delta).Sample()
            | ContinuousUniform delta -> ContinuousUniform(input - delta, input + delta).Sample()
            | PositiveContinousUniform delta -> 
                let u = ContinuousUniform(input - delta, input + delta).Sample()
                if u <= 0. then input + 0.1 else u

        let step (iteration : int) : AcceptanceRejection =
            let proposed           = matchedProposalDistribution current
            let currentProbability = simpleBayesianModel.GetPosteriorWithoutScalingFactor current request.PrintDebug
            let proposedProbabilty = simpleBayesianModel.GetPosteriorWithoutScalingFactor proposed request.PrintDebug
            let acceptanceRatio    = Math.Min(currentProbability / proposedProbabilty, 1.)
            let uniformDraw        = zeroOneUniform.Sample()

            if request.PrintDebug then 
                printfn "Chain: %A Iteration: %A - Current Probability: %A | Proposed Probability: %A | AcceptanceRatio: %A | Uniform Draw: %A | Current: %A | Proposed: %A" 
                    id iteration currentProbability proposedProbabilty acceptanceRatio uniformDraw current proposed

            if uniformDraw < acceptanceRatio then (current <- proposed; Acceptance proposed) 
            else Rejection current
    
        let stepResults : AcceptanceRejection seq =
            seq {1..iterations}
            |> Seq.map step
            |> Seq.skip burnin

        let getAcceptanceRateAndStepValues : float * float seq =

            // Compute the Acceptance Rate.
            let acceptanceRate : float = 
                let totalBurninWithoutBurnin : float = float (Seq.length stepResults)
                let totalNumberOfAcceptances : float = 
                    stepResults 
                    |> Seq.filter(fun x -> 
                        match x with
                        | Acceptance x -> true
                        | _ -> false)
                    |> Seq.length
                    |> float
                totalNumberOfAcceptances / totalBurninWithoutBurnin

            // Grab the Step Values that'll approximate the posterior.
            let stepValues =
                stepResults |> Seq.map(fun s ->
                    match s with
                    | Acceptance v -> v
                    | Rejection v -> v)

            acceptanceRate, stepValues

        let acceptanceRate, stepValues = getAcceptanceRateAndStepValues 

        { Id             = id
          AcceptanceRate = acceptanceRate
          StepValues     = stepValues} 

    let chains : MCMCChain seq =
        seq {1..request.Chains}
        |> Seq.map(fun id ->  getChain id request iterations simpleBayesianModel)

    { Chains      = chains
      MCMCRequest =  request }

### Running the Algorithm

In [ ]:
let runMCMC (request : MCMCRequest) 
            (model : SimpleBayesianNetworkModel) : MCMCResult =
    match request.StepFunction with
    | SymmetricMetropolisHastings (proposalDistribution,initialValue) ->
        match request.ConvergenceCriteria with 
        | IterativeConvergence iterations -> 
            doSymmetricMetropolisHastings request iterations proposalDistribution initialValue model
        | _ -> failwith "You need to pass in the number of iterations for the Metropolis-Hastings algorithm"
    | _ -> failwithf "Step Function Not Registered: %A" request.StepFunction

### Tieing it All Together

#### Exponential Prior and Exponential Likelihood

In [ ]:
let model = @"x ~ Exponential(a) 
              y|x ~ Exponential(x) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {a : 2., b : 2.}, observed : [1,2,3,4,4,2,5,6,7,3,2,3,4,5,6,1,2,3,4,4,4,4]}"
let simpleModel = SimpleBayesianNetworkModel.ConstructModel "Exponential Model" parsedModel (deserializeParameters paramList)

let request : MCMCRequest = 
    { StepFunction        = SymmetricMetropolisHastings (ProposalDistribution.PositiveContinousUniform 0.002, 0.001)
      ConvergenceCriteria = IterativeConvergence 10000
      BurnInIterationsPct = 0.2
      PrintDebug          = false 
      Chains              = 4 }

let mcmc       = runMCMC request simpleModel
let firstChain = Seq.head mcmc.Chains

Histogram(x = firstChain.StepValues)
|> Chart.Plot
|> Chart.WithTitle "Exponential Prior and Likelihood = Exponential Posterior"

#### Real Life Example: Normal Prior and Normal Likelihood

In [8]:
#!fsharp
let model = @"x ~ Normal(μ,τ) 
              y|x ~ Exponential(x) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {μ : 5, τ : 3.1622, σ : 1}, observed : [9.37,10.18,9.16,11.60,10.33]}"
let simpleModel = 
    SimpleBayesianNetworkModel.ConstructModel "Normal-Exponential" parsedModel (deserializeParameters paramList)


let request : MCMCRequest = 
    { StepFunction        = SymmetricMetropolisHastings (ProposalDistribution.PositiveContinousUniform 0.0003, 0.0001)
      ConvergenceCriteria = IterativeConvergence 5000
      BurnInIterationsPct = 0.2
      PrintDebug          = false 
      Chains              = 4 }

let mcmc       = runMCMC request simpleModel
let firstChain = Seq.head mcmc.Chains

printfn "Acceptance Rate: %A" firstChain.AcceptanceRate
Histogram(x = firstChain.StepValues)
|> Chart.Plot
|> Chart.WithTitle "Normal"
|> Chart.WithWidth 700
|> Chart.WithHeight 500

Unhandled Exception: input.fsx (7,5)-(7,31) typecheck error The value, namespace, type or module 'SimpleBayesianNetworkModel' is not defined.
input.fsx (10,15)-(10,26) typecheck error The type 'MCMCRequest' is not defined.
input.fsx (11,7)-(11,19) typecheck error The record label 'StepFunction' is not defined.
input.fsx (17,18)-(17,25) typecheck error The value or constructor 'runMCMC' is not defined. Maybe you want one of the following:
   Rune
input.fsx (18,27)-(18,38) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (20,31)-(20,56) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx 

## Conclusion

This submission took a lot of work and is a product of two months of going down a Bayesian rabbit hole. Since I think I have built a relatively strong base, I will continue to build on top of this. As an obvious disclaimer, none of this code should be used in production - some of it is untested and might have bugs that weren't apparent to me while writing this. Also, please let me know if you find any mistakes! I'll be happy to learn from them and fix up this notebook.

Thanks to the organizers of FSharp-Advent and Happy Holidays to all!

# Lessons and Prospective Improvements

## Lessons

### Debugging an Iterative Algorithm Can Be Challenging

### Tooling And Workflow is of Paramount Importance

### Prototyping Helps With Creating Modular Functions

## Improvements

### Better Parameterization

### More Generalized Bayesian Network

### More Efficient MCMC Methods

## References

## Music Listened to While Working On This Project

1. [John Zorn and Friends](https://www.youtube.com/watch?v=c4eO2o9u1j0&ab_channel=podgoryt)
2. [Mike Patton and Mondo Cane](https://www.youtube.com/watch?v=iDOl5q7UXfg&ab_channel=FNM4EVER2)
3. [Burt Bacharach - This Guy's In Love with you](https://www.youtube.com/watch?v=2dDGnl8_Dzg&ab_channel=MusicWonders )
4. [Melvins - Lysol](https://www.youtube.com/watch?v=JtO_Awk4pqU&ab_channel=AboveDeath)
5. [The Jesus Lizard - Puss](https://www.youtube.com/watch?v=PaBJQG6A9SQ&ab_channel=spycory1)
6. [Mr. Bungle - Sudden Death](https://www.youtube.com/watch?v=-QWFV_057KM&ab_channel=IpecacRecordings)
7. [Madreblu - Certamente](https://www.youtube.com/watch?v=Ivh0FWTSJ78&ab_channel=Milano2000Records)
8. [Pato Banton - Spirits in the Material World](https://www.youtube.com/watch?v=S--kTyvm_fM&ab_channel=PatoBanton%26TheReggaeRevolution-Topic)
9. [Max Cooper - Ripple](https://www.youtube.com/watch?v=P_X1KGCgWlE&ab_channel=MaxCooper-Topic)
10. [The Sword - Cheap Sunglasses](https://www.youtube.com/watch?v=RYnTLIZkjlg&ab_channel=SwordofDoomMusic)
11. [Brother Dege - Too Old To Die Young](https://www.youtube.com/watch?v=FQNFcYvILJE&ab_channel=UltimatePowa)
12. [Mr. Bungle - Retrovertigo](https://www.youtube.com/watch?v=DRyh2cxJCp0&ab_channel=tkan)